In [18]:
### Imports
import pandas as pd
import numpy as np
import json

In [19]:
# Step 1: Load the JSON Data
with open("sessions.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Step 2: Extract Relevant Information
callouts_data = []

for session in data:
    for section in session["sections"]:
        for callout in section["callouts"]:
            if "caller" in callout:  # Ensure there's a caller identified
                callouts_data.append(
                    {
                        "date": session["date"],
                        "period": session["period"],
                        "session": session["sessionNumber"],
                        "speaker_id": section["speaker"],
                        "caller_id": callout.get("caller"),
                    }
                )

# Step 3: Create the DataFrame
df_callouts = pd.DataFrame(callouts_data)

# Display the first few rows of the DataFrame to verify
print(df_callouts.head())

                  date period  session speaker_id caller_id
0  2023-09-20T00:00:00  XXVII      230      14795     52687
1  2023-09-20T00:00:00  XXVII      230      14795     35514
2  2023-09-20T00:00:00  XXVII      230      14795     35514
3  2023-09-20T00:00:00  XXVII      230       6486     35520
4  2023-09-20T00:00:00  XXVII      230       6486     35520


In [33]:
# Load the persons data ('persons.json')
with open("persons.json", "r", encoding="utf-8") as file:
    persons_data = json.load(file)

# Create the lookup tables
persons_lookup = {
    person["id"]: {"name": person["name"], "parties": person["parties"]}
    for person in persons_data
}

# Update the DataFrame with caller_name, speaker_name, caller_parties, and speaker_parties
df_callouts["caller_name"] = df_callouts["caller_id"].map(
    lambda x: persons_lookup.get(x, {}).get("name", "Unknown")
)
df_callouts["speaker_name"] = df_callouts["speaker_id"].map(
    lambda x: persons_lookup.get(x, {}).get("name", "Unknown")
)
df_callouts["caller_parties"] = df_callouts["caller_id"].map(
    lambda x: ", ".join(persons_lookup.get(x, {}).get("parties", []))
)
df_callouts["speaker_parties"] = df_callouts["speaker_id"].map(
    lambda x: ", ".join(persons_lookup.get(x, {}).get("parties", []))
)

# Display the updated DataFrame
print(df_callouts.head())

                  date period  session speaker_id caller_id  \
0  2023-09-20T00:00:00  XXVII      230      14795     52687   
1  2023-09-20T00:00:00  XXVII      230      14795     35514   
2  2023-09-20T00:00:00  XXVII      230      14795     35514   
3  2023-09-20T00:00:00  XXVII      230       6486     35520   
4  2023-09-20T00:00:00  XXVII      230       6486     35520   

             caller_name            speaker_name caller_parties  \
0  Alois Stöger, diplômé         August Wöginger            SPÖ   
1        Wolfgang Zanger         August Wöginger            FPÖ   
2        Wolfgang Zanger         August Wöginger            FPÖ   
3          Herbert Kickl  MMag. Dr. Susanne Raab            FPÖ   
4          Herbert Kickl  MMag. Dr. Susanne Raab            FPÖ   

  speaker_parties  
0             ÖVP  
1             ÖVP  
2             ÖVP  
3             ÖVP  
4             ÖVP  


In [34]:
df_callouts.to_csv(
    "df_callouts_total.tsv", sep="\t", index=False
)  # save to TSV because initial parsing takes long

In [48]:
df_callouts_XXVII = df_callouts[
    (df_callouts["period"] == "XXVII") & (df_callouts["session"] == 230) #also filter for a specific session to not overcrowd the data
]
# find people with multiple party asociations and manually correct for most current afiliation
multiple_parties = (df_callouts["caller_parties"].str.contains(",", na=False)) | (
    df_callouts["speaker_parties"].str.contains(",", na=False)
)

party_corrections = {"2345": "GRÜNE", "51577": "FPÖ", "2867": "FPÖ"}

for id_str, correct_party in party_corrections.items():
    # Check for matches in 'caller_id' and update 'caller_parties'
    match_caller = df_callouts_XXVII["caller_id"] == id_str
    df_callouts_XXVII.loc[match_caller, "caller_parties"] = correct_party

    # Check for matches in 'speaker_id' and update 'speaker_parties'
    match_speaker = df_callouts_XXVII["speaker_id"] == id_str
    df_callouts_XXVII.loc[match_speaker, "speaker_parties"] = correct_party

df_callouts_XXVII.replace("", np.nan, inplace=True)
df_callouts_XXVII

/tmp/ipykernel_1281/129990645.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callouts_XXVII.replace("", np.nan, inplace=True)


,date,period,session,speaker_id,caller_id,caller_name,speaker_name,caller_parties,speaker_parties
0,2023-09-20T00:00:00,XXVII,230,14795,52687,"Alois Stöger, diplômé",August Wöginger,SPÖ,ÖVP
1,2023-09-20T00:00:00,XXVII,230,14795,35514,Wolfgang Zanger,August Wöginger,FPÖ,ÖVP
2,2023-09-20T00:00:00,XXVII,230,14795,35514,Wolfgang Zanger,August Wöginger,FPÖ,ÖVP
3,2023-09-20T00:00:00,XXVII,230,6486,35520,Herbert Kickl,MMag. Dr. Susanne Raab,FPÖ,ÖVP
4,2023-09-20T00:00:00,XXVII,230,6486,35520,Herbert Kickl,MMag. Dr. Susanne Raab,FPÖ,ÖVP
...,...,...,...,...,...,...,...,...,...
651,2023-09-20T00:00:00,XXVII,230,5652,78586,"Christian Hafenecker, MA",Mag. Ulrike Fischer,FPÖ,GRÜNE
652,2023-09-20T00:00:00,XXVII,230,5652,5678,Mag. Nina Tomaselli,Mag. Ulrike Fischer,GRÜNE,GRÜNE
653,2023-09-20T00:00:00,XXVII,230,5652,78586,"Christian Hafenecker, MA",Mag. Ulrike Fischer,FPÖ,GRÜNE
654,2023-09-20T00:00:00,XXVII,230,20281,35489,Franz Hörl,"MMag. Katharina Werner, Bakk.",ÖVP,NEOS


In [50]:
# Add a prefix to caller_id and speaker_id
df_callouts_XXVII['caller_id'] = 'c' + df_callouts_XXVII['caller_id'].astype(str)
df_callouts_XXVII['speaker_id'] = 's' + df_callouts_XXVII['speaker_id'].astype(str)

df_callouts_XXVII_grouped = (
    df_callouts_XXVII.groupby(
        [
            "caller_id",
            "speaker_id",
            "caller_name",
            "speaker_name",
            "caller_parties",
            "speaker_parties",
        ]
    )
    .size()
    .reset_index(name="counts")
)

# Assuming df_callouts_XXVII_grouped is your starting DataFrame
# First, calculate the total counts for each caller_id
total_counts_per_caller = df_callouts_XXVII_grouped.groupby('caller_id')['counts'].sum().reset_index(name='counts_total')

# Then, merge this total counts back into your original (or grouped) DataFrame
df_callouts_XXVII_grouped_with_totals = pd.merge(
    df_callouts_XXVII_grouped,
    total_counts_per_caller,
    on='caller_id',
    how='left'
)

# Sorting the DataFrame based on counts_total (you might also want to keep the original sorting as a secondary criterion)
df_sorted_by_total_counts = df_callouts_XXVII_grouped_with_totals.sort_values(by=['counts_total', 'counts'], ascending=False)


df_sorted_by_total_counts.to_csv(
    "df_callouts_XXVII.tsv", sep="\t", index=False
)  # save data as final .tsv

df_sorted_by_total_counts

/tmp/ipykernel_1281/4211430033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callouts_XXVII['caller_id'] = 'c' + df_callouts_XXVII['caller_id'].astype(str)
/tmp/ipykernel_1281/4211430033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callouts_XXVII['speaker_id'] = 's' + df_callouts_XXVII['speaker_id'].astype(str)


,caller_id,speaker_id,caller_name,speaker_name,caller_parties,speaker_parties,counts,counts_total
141,cc35520,ss2136,Herbert Kickl,"Karl Nehammer, MSc",FPÖ,ÖVP,11,61
143,cc35520,ss22694,Herbert Kickl,Mag. Jörg Leichtfried,FPÖ,SPÖ,7,61
150,cc35520,ss5439,Herbert Kickl,Dr. Christian Stocker,FPÖ,ÖVP,7,61
153,cc35520,ss5672,Herbert Kickl,Barbara Neßler,FPÖ,GRÜNE,5,61
156,cc35520,ss6486,Herbert Kickl,MMag. Dr. Susanne Raab,FPÖ,ÖVP,5,61
...,...,...,...,...,...,...,...,...
267,cc72999,ss83137,Mag. Klaus Fürlinger,Peter Wurm,ÖVP,FPÖ,1,1
288,cc8235,ss30653,Nikolaus Prinz,"Melanie Erasim, MSc",ÖVP,SPÖ,1,1
311,cc83124,ss5486,Michael Bernhard,Laurenz Pöttinger,NEOS,ÖVP,1,1
359,cc83299,ss80479,Ing. Manfred Hofinger,Mag. Karin Greiner,ÖVP,SPÖ,1,1
